In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append(os.path.realpath('..'))

In [24]:
import textwrap

import numpy as np
import pandas as pd
from scipy.stats import chi2

from sympy.solvers import solve, solveset, nsolve, nonlinsolve
from sympy import Symbol, symbols, N, sin, cos
from sympy.matrices import Matrix, MatrixSymbol
from sympy.abc import a, b, c, d

import plotly.express as ex
import plotly.graph_objects as go

from tracking_v2.kalman.turn import CoordinatedTurn
from tracking_v2.sensor import GeometricSensor
from tracking_v2.target import SingleTurnTarget, SinusTarget
from tracking.util import to_df

In [4]:
Omega, vx, vy, T = symbols("\Omega, v_x, v_y, T")
Px, Py, Pvx, Pvy, Pomega = symbols("P_x, P_y, P_vx, P_vy, P_\omega", positive=True)
Rx, Ry = symbols("R_x, R_y", positive=True)
Qx, Qy, Qomega = symbols("Q_x, Q_y, Q_\Omega", positive=True)

T = 1

In [5]:
ST = sin(Omega * T)
CT = cos(Omega * T)
XT = vx * T
YT = vy * T
O2 = Omega*Omega

f_Omega_1 = (CT * XT)/Omega - \
            (ST * vx)/O2 - \
            (ST * YT)/Omega - \
            ((-1 + CT)*vy)/O2

f_Omega_2 = -ST*XT - CT*YT

f_Omega_3 = (ST * XT)/Omega - \
            ((1-CT)*vx)/O2 + \
            (CT*YT)/Omega - \
            (ST*vy)/O2

f_Omega_4 = CT*XT - ST*YT

f_x = Matrix([[1, sin(Omega * T)/Omega, 0, -(1-cos(Omega * T))/Omega, f_Omega_1],
              [0, cos(Omega * T), 0, -sin(Omega * T), f_Omega_2],
              [0, (1-cos(Omega * T))/Omega, 1, sin(Omega * T)/Omega, f_Omega_3],
              [0, sin(Omega * T), 0, cos(Omega * T), f_Omega_4],
              [0, 0, 0, 0, 1]])

P = Matrix([[Px, 0, 0, 0, 0],
            [0, Pvx, 0, 0, 0],
            [0, 0, Py, 0, 0],
            [0, 0, 0, Pvy, 0],
            [0, 0, 0, 0, Pomega]])

H = Matrix([[1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0]])

R = Matrix([[Rx, 0],
            [0, Ry]])

Q = Matrix([[T*T/2*Qx, 0, 0, 0, 0],
            [0, T*Qx, 0, 0, 0],
            [0, 0, T*T/2*Qy, 0, 0],
            [0, 0, 0, T*Qy, 0],
            [0, 0, 0, 0, T*Qomega]])

In [6]:
f_x

Matrix([
[1,       sin(\Omega)/\Omega, 0, (cos(\Omega) - 1)/\Omega, v_x*cos(\Omega)/\Omega - v_y*sin(\Omega)/\Omega - v_x*sin(\Omega)/\Omega**2 - v_y*(cos(\Omega) - 1)/\Omega**2],
[0,              cos(\Omega), 0,             -sin(\Omega),                                                                            -v_x*sin(\Omega) - v_y*cos(\Omega)],
[0, (1 - cos(\Omega))/\Omega, 1,       sin(\Omega)/\Omega, v_x*sin(\Omega)/\Omega + v_y*cos(\Omega)/\Omega - v_x*(1 - cos(\Omega))/\Omega**2 - v_y*sin(\Omega)/\Omega**2],
[0,              sin(\Omega), 0,              cos(\Omega),                                                                             v_x*cos(\Omega) - v_y*sin(\Omega)],
[0,                        0, 0,                        0,                                                                                                             1]])

In [7]:
P

Matrix([
[P_x,    0,   0,    0,        0],
[  0, P_vx,   0,    0,        0],
[  0,    0, P_y,    0,        0],
[  0,    0,   0, P_vy,        0],
[  0,    0,   0,    0, P_\omega]])

In [8]:
f_x @ P @ f_x.T + Q

Matrix([
[                                                                                                                     P_\omega*(v_x*cos(\Omega)/\Omega - v_y*sin(\Omega)/\Omega - v_x*sin(\Omega)/\Omega**2 - v_y*(cos(\Omega) - 1)/\Omega**2)**2 + P_vx*sin(\Omega)**2/\Omega**2 + P_vy*(cos(\Omega) - 1)**2/\Omega**2 + P_x + 0.5*Q_x, P_\omega*(-v_x*sin(\Omega) - v_y*cos(\Omega))*(v_x*cos(\Omega)/\Omega - v_y*sin(\Omega)/\Omega - v_x*sin(\Omega)/\Omega**2 - v_y*(cos(\Omega) - 1)/\Omega**2) + P_vx*sin(\Omega)*cos(\Omega)/\Omega - P_vy*(cos(\Omega) - 1)*sin(\Omega)/\Omega, P_\omega*(v_x*sin(\Omega)/\Omega + v_y*cos(\Omega)/\Omega - v_x*(1 - cos(\Omega))/\Omega**2 - v_y*sin(\Omega)/\Omega**2)*(v_x*cos(\Omega)/\Omega - v_y*sin(\Omega)/\Omega - v_x*sin(\Omega)/\Omega**2 - v_y*(cos(\Omega) - 1)/\Omega**2) + P_vx*(1 - cos(\Omega))*sin(\Omega)/\Omega**2 + P_vy*(cos(\Omega) - 1)*sin(\Omega)/\Omega**2,          P_\omega*(v_x*cos(\Omega) - v_y*sin(\Omega))*(v_x*cos(\Omega)/\Omega - v_y*sin(\Omeg

In [9]:
Matrix

sympy.matrices.dense.MutableDenseMatrix

In [10]:
F   = symbols('F0:12')
f_x = Matrix([[1, F[0], 0, F[4], F[8]],
              [0, F[1], 0, F[5], F[9]],
              [0, F[2], 1, F[6], F[10]],
              [0, F[3], 0, F[7], F[11]],
              [0, 0, 0, 0, 1]])

P = Matrix([[Px, 0, 0, 0, 0],
            [0, Pvx, 0, 0, 0],
            [0, 0, Py, 0, 0],
            [0, 0, 0, Pvy, 0],
            [0, 0, 0, 0, Pomega]])

H = Matrix([[1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0]])

R = Matrix([[Rx, 0],
            [0, Ry]])

Q = Matrix([[T*T/2*Qx, 0, 0, 0, 0],
            [0, T*Qx, 0, 0, 0],
            [0, 0, T*T/2*Qy, 0, 0],
            [0, 0, 0, T*Qy, 0],
            [0, 0, 0, 0, T*Qomega]])

In [11]:
P_1 = f_x @ P @ f_x.T + Q
S = H @ P_1 @ H.T + R
K = P_1 @ H.T @ S.inv()

In [12]:
K.simplify()

In [13]:
K

Matrix([
[                           (-1.0*(F0*F2*P_vx + F10*F8*P_\omega + F4*F6*P_vy)**2 + (F0**2*P_vx + F4**2*P_vy + F8**2*P_\omega + P_x + 0.5*Q_x)*(1.0*F10**2*P_\omega + 1.0*F2**2*P_vx + 1.0*F6**2*P_vy + 1.0*P_y + 0.5*Q_y + 1.0*R_y))/(1.0*F0**2*F10**2*P_\omega*P_vx + 1.0*F0**2*F6**2*P_vx*P_vy + 1.0*F0**2*P_vx*P_y + 0.5*F0**2*P_vx*Q_y + 1.0*F0**2*P_vx*R_y - 2.0*F0*F10*F2*F8*P_\omega*P_vx - 2.0*F0*F2*F4*F6*P_vx*P_vy + 1.0*F10**2*F4**2*P_\omega*P_vy + 1.0*F10**2*P_\omega*P_x + 0.5*F10**2*P_\omega*Q_x + 1.0*F10**2*P_\omega*R_x - 2.0*F10*F4*F6*F8*P_\omega*P_vy + 1.0*F2**2*F4**2*P_vx*P_vy + 1.0*F2**2*F8**2*P_\omega*P_vx + 1.0*F2**2*P_vx*P_x + 0.5*F2**2*P_vx*Q_x + 1.0*F2**2*P_vx*R_x + 1.0*F4**2*P_vy*P_y + 0.5*F4**2*P_vy*Q_y + 1.0*F4**2*P_vy*R_y + 1.0*F6**2*F8**2*P_\omega*P_vy + 1.0*F6**2*P_vy*P_x + 0.5*F6**2*P_vy*Q_x + 1.0*F6**2*P_vy*R_x + 1.0*F8**2*P_\omega*P_y + 0.5*F8**2*P_\omega*Q_y + 1.0*F8**2*P_\omega*R_y + 1.0*P_x*P_y + 0.5*P_x*Q_y + 1.0*P_x*R_y + 0.5*P_y*Q_x + 1.0*P_y*R_x + 0.25*Q

In [15]:
f_x @ P

Matrix([
[P_x, F0*P_vx,   0, F4*P_vy,  F8*P_\omega],
[  0, F1*P_vx,   0, F5*P_vy,  F9*P_\omega],
[  0, F2*P_vx, P_y, F6*P_vy, F10*P_\omega],
[  0, F3*P_vx,   0, F7*P_vy, F11*P_\omega],
[  0,       0,   0,       0,     P_\omega]])

In [16]:
P @ f_x.T

Matrix([
[        P_x,           0,            0,            0,        0],
[    F0*P_vx,     F1*P_vx,      F2*P_vx,      F3*P_vx,        0],
[          0,           0,          P_y,            0,        0],
[    F4*P_vy,     F5*P_vy,      F6*P_vy,      F7*P_vy,        0],
[F8*P_\omega, F9*P_\omega, F10*P_\omega, F11*P_\omega, P_\omega]])

In [18]:
P_1.simplify()
P_1

Matrix([
[F0**2*P_vx + F4**2*P_vy + F8**2*P_\omega + P_x + 0.5*Q_x,       F0*F1*P_vx + F4*F5*P_vy + F8*F9*P_\omega,                 F0*F2*P_vx + F10*F8*P_\omega + F4*F6*P_vy,       F0*F3*P_vx + F11*F8*P_\omega + F4*F7*P_vy,         F8*P_\omega],
[                F0*F1*P_vx + F4*F5*P_vy + F8*F9*P_\omega, F1**2*P_vx + F5**2*P_vy + F9**2*P_\omega + Q_x,                 F1*F2*P_vx + F10*F9*P_\omega + F5*F6*P_vy,       F1*F3*P_vx + F11*F9*P_\omega + F5*F7*P_vy,         F9*P_\omega],
[               F0*F2*P_vx + F10*F8*P_\omega + F4*F6*P_vy,      F1*F2*P_vx + F10*F9*P_\omega + F5*F6*P_vy, F10**2*P_\omega + F2**2*P_vx + F6**2*P_vy + P_y + 0.5*Q_y,      F10*F11*P_\omega + F2*F3*P_vx + F6*F7*P_vy,        F10*P_\omega],
[               F0*F3*P_vx + F11*F8*P_\omega + F4*F7*P_vy,      F1*F3*P_vx + F11*F9*P_\omega + F5*F7*P_vy,                F10*F11*P_\omega + F2*F3*P_vx + F6*F7*P_vy, F11**2*P_\omega + F3**2*P_vx + F7**2*P_vy + Q_y,        F11*P_\omega],
[                                          

In [19]:
H @ P_1 @ H.T

Matrix([
[F0**2*P_vx + F4**2*P_vy + F8**2*P_\omega + P_x + 0.5*Q_x,                 F0*F2*P_vx + F10*F8*P_\omega + F4*F6*P_vy],
[               F0*F2*P_vx + F10*F8*P_\omega + F4*F6*P_vy, F10**2*P_\omega + F2**2*P_vx + F6**2*P_vy + P_y + 0.5*Q_y]])

In [21]:
(H @ P_1 @ H.T).inv()

Matrix([
[(1.0*F10**2*P_\omega + 1.0*F2**2*P_vx + 1.0*F6**2*P_vy + 1.0*P_y + 0.5*Q_y)/(1.0*F0**2*F10**2*P_\omega*P_vx + 1.0*F0**2*F6**2*P_vx*P_vy + 1.0*F0**2*P_vx*P_y + 0.5*F0**2*P_vx*Q_y - 2.0*F0*F10*F2*F8*P_\omega*P_vx - 2.0*F0*F2*F4*F6*P_vx*P_vy + 1.0*F10**2*F4**2*P_\omega*P_vy + 1.0*F10**2*P_\omega*P_x + 0.5*F10**2*P_\omega*Q_x - 2.0*F10*F4*F6*F8*P_\omega*P_vy + 1.0*F2**2*F4**2*P_vx*P_vy + 1.0*F2**2*F8**2*P_\omega*P_vx + 1.0*F2**2*P_vx*P_x + 0.5*F2**2*P_vx*Q_x + 1.0*F4**2*P_vy*P_y + 0.5*F4**2*P_vy*Q_y + 1.0*F6**2*F8**2*P_\omega*P_vy + 1.0*F6**2*P_vy*P_x + 0.5*F6**2*P_vy*Q_x + 1.0*F8**2*P_\omega*P_y + 0.5*F8**2*P_\omega*Q_y + 1.0*P_x*P_y + 0.5*P_x*Q_y + 0.5*P_y*Q_x + 0.25*Q_x*Q_y),                   (-1.0*F0*F2*P_vx - 1.0*F10*F8*P_\omega - 1.0*F4*F6*P_vy)/(1.0*F0**2*F10**2*P_\omega*P_vx + 1.0*F0**2*F6**2*P_vx*P_vy + 1.0*F0**2*P_vx*P_y + 0.5*F0**2*P_vx*Q_y - 2.0*F0*F10*F2*F8*P_\omega*P_vx - 2.0*F0*F2*F4*F6*P_vx*P_vy + 1.0*F10**2*F4**2*P_\omega*P_vy + 1.0*F10**2*P_\omega*P_x + 0.5*F10

In [20]:
P_1 @ H.T

Matrix([
[F0**2*P_vx + F4**2*P_vy + F8**2*P_\omega + P_x + 0.5*Q_x,                 F0*F2*P_vx + F10*F8*P_\omega + F4*F6*P_vy],
[                F0*F1*P_vx + F4*F5*P_vy + F8*F9*P_\omega,                 F1*F2*P_vx + F10*F9*P_\omega + F5*F6*P_vy],
[               F0*F2*P_vx + F10*F8*P_\omega + F4*F6*P_vy, F10**2*P_\omega + F2**2*P_vx + F6**2*P_vy + P_y + 0.5*Q_y],
[               F0*F3*P_vx + F11*F8*P_\omega + F4*F7*P_vy,                F10*F11*P_\omega + F2*F3*P_vx + F6*F7*P_vy],
[                                             F8*P_\omega,                                              F10*P_\omega]])

In [25]:
Matrix([[a, b], [c, d]]).inv()

Matrix([
[ d/(a*d - b*c), -b/(a*d - b*c)],
[-c/(a*d - b*c),  a/(a*d - b*c)]])